# Netmiko & Diffing

In [1]:
# Local CSR 1000v (running under vagrant) -- rtr1
HOST = '127.0.0.1'
PORT_NC = 2223
PORT_SSH = 2222
USER = 'vagrant'
PASS = 'vagrant'
PLATFORM = 'cisco_ios'

## Connect both netmiko and ncclient

In [2]:
from netmiko import ConnectHandler
from ncclient import manager
from lxml import etree


def pretty_print(retval):
    print(etree.tostring(retval.data, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True

def get_reply(chan, eom):
    bytes = u''
    while bytes.find(eom)==-1:
        bytes += chan.recv(65535).decode('utf-8')
    return bytes

m = manager.connect(host=HOST, port=PORT_NC, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)
d = ConnectHandler(device_type=PLATFORM, ip=HOST, port=PORT_SSH, username=USER, password=PASS)

prompt = d.find_prompt()

## Sample Config To Apply

In [3]:
config = '''ip access-list standard FOO
 permit 10.128.0.0 0.127.255.255
 permit 10.119.120.0 0.0.7.255
 permit 10.87.79.0 0.0.0.255
 permit 10.87.96.0 0.0.0.255
 deny   any
'''

## Copy Config To Flash (IOS-XE Only)

In [4]:
d.remote_conn.sendall('copy running-config flash:backup123.cfg\n')
get_reply(d.remote_conn, ']? ')
d.remote_conn.sendall('\n')
output = get_reply(d.remote_conn, prompt)

## Apply Config

In [5]:
output = d.send_config_set(config.splitlines())

## Capture Running Config

In [6]:
running_config_before = d.send_command('show running-config')

## Capture Netconf Config

In [7]:
retval = m.get_config(source='running', filter=('xpath', '/native'))
native = retval.data_ele[0]
reapply = '<config>' + etree.tostring(native) + '</config>'

## Restore Config From Flash

In [8]:
d.remote_conn.sendall('configure replace flash:backup123.cfg\n')
output = get_reply(d.remote_conn, '[no]: ')
print(output)
d.remote_conn.sendall('y\n')
output = get_reply(d.remote_conn, prompt)
print(output)

configure replace flash:backup123.cfg
This will apply all necessary additions and deletions
to replace the current running configuration with the
contents of the specified configuration file, which is
assumed to be a complete configuration, not a partial
configuration. Enter Y if you are sure you want to proceed. ? [no]: 
y
Total number of passes: 0
Rollback Done

csr1kv#


## Reapply Netconf Config

In [9]:
m.edit_config(reapply, format='xml', target='running', default_operation='merge')

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:653df30f-437c-4086-ad34-575e96289ad3" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

## Get Running Config Again

In [10]:
running_config_after = d.send_command('show running-config')

## Diff Before & After

In [11]:
from difflib import context_diff

#
# remember to skip the first few lines that have timestamps & stuff that may differ
#
print(''.join(context_diff(running_config_before.splitlines()[5:], running_config_after.splitlines()[5:])))

## Delete Backup From Flash

In [12]:
d.remote_conn.sendall('delete flash:backup123.cfg\n')
output = get_reply(d.remote_conn, ']? ')
d.remote_conn.sendall('\n')
output = get_reply(d.remote_conn, '[confirm]')
d.remote_conn.sendall('\n')
output = get_reply(d.remote_conn, prompt)

## Tidyup Sessions

In [13]:
d.disconnect()
m.close_session()

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:d413420d-396a-4fbb-ac63-1c99a1434550" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>